# Course: ICS 574
# Prject: Alchemists Project - EDA
- Date: May 1st 2024
- Project Members
    - AHMED DHAFER ALQARNI, ID: 201453160
    - WALEED ABDULLAH ALFAIFI, ID: 201640920
    - ALGHAMDI, BANDAR, ID: 202206560

In [1]:
# import python needed for the project
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# import the datasets as dataframes
stocks_df = pd.read_csv("./datasets/Tadawul_stcks.csv")
brent_df = pd.read_csv("./datasets/BrentOilPrices.csv")
eth_df = pd.read_csv("./datasets/coin_Ethereum.csv")
gold_df = pd.read_csv("./datasets/Gold_Daily.csv")

In [2]:
# for the merged dataframes, perform the following operations
# 1. experimental goal
# 2. experiment variables
# 3. comparative study
# 4. depth of results analysis
# 5. evaluation measures
# 6. threats to validity

# 1: Experimental Goal

In [2]:
# 1. experimental goal

# perform the following operations on the dataframes (similar to the EDA part)
stocks_specific_df = stocks_df[(stocks_df['sectoer'] == 'Energy') | (stocks_df['sectoer'] == 'Materials')]
stocks_specific_df.rename(columns={'date': 'Date'}, inplace=True)
stocks_specific_df.rename(columns={'close': 'Stock_Price'}, inplace=True)
stocks_specific_df['Date'] = pd.to_datetime(stocks_specific_df['Date'])
brent_df['Date'] = pd.to_datetime(brent_df['Date'])
gold_df['Date'] = pd.to_datetime(gold_df['Date'])
brent_df.rename(columns={'Price': 'Brent_Price'}, inplace=True)
gold_df.rename(columns={'Price': 'Gold_Price'}, inplace=True)
stocks_specific_df.rename(columns={'sectoer': 'Sector'}, inplace=True)
stocks_specific_df = stocks_specific_df[['Date', 'name', 'Stock_Price', 'Sector', 'symbol']]
brent_df = brent_df[['Date', 'Brent_Price']]
gold_df = gold_df[['Date', 'Gold_Price']]
stocks_specific_df.fillna(method='ffill', inplace=True)
stocks_brent_df = pd.merge(stocks_specific_df, brent_df, on='Date', how='inner')
stocks_brent_gold_df = pd.merge(stocks_brent_df, gold_df, on='Date', how='inner')


/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_18587/1369400171.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_specific_df.rename(columns={'date': 'Date'}, inplace=True)
/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_18587/1369400171.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_specific_df.rename(columns={'close': 'Stock_Price'}, inplace=True)
/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_18587/1369400171.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [3]:
stocks_brent_gold_df['Brent_Price'] = stocks_brent_gold_df['Brent_Price'] * 3.75
stocks_brent_gold_df['Gold_Price'] = stocks_brent_gold_df['Gold_Price'] * 3.75

In [4]:
print("Merged Dataset Missing Values:")
print(stocks_brent_gold_df.isnull().sum())

Merged Dataset Missing Values:
Date           0
name           0
Stock_Price    0
Sector         0
symbol         0
Brent_Price    0
Gold_Price     0
dtype: int64


# 2: Experiment Variables

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
# create a spark session and display spark session information
# notice local[*] means run spark locally with as all available cores & memory on the local machine to perform parallel computing
spark = SparkSession.builder.appName("Decision Tree Classifier Example").getOrCreate()
spark

24/03/27 09:42:44 WARN Utils: Your hostname, Bandars-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.154 instead (on interface en0)
24/03/27 09:42:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/27 09:42:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
print("Merged Dataset columns:")
print(stocks_brent_gold_df.columns)

Merged Dataset columns:
Index(['Date', 'name', 'Stock_Price', 'Sector', 'symbol', 'Brent_Price',
       'Gold_Price'],
      dtype='object')


In [8]:
# change the pandas dataframe to a spark dataframe
stocks_brent_gold_df_spark = spark.createDataFrame(stocks_brent_gold_df)
stocks_brent_gold_df_spark.show()

+-------------------+--------------------+-----------+---------+------+-----------+----------+
|               Date|                name|Stock_Price|   Sector|symbol|Brent_Price|Gold_Price|
+-------------------+--------------------+-----------+---------+------+-----------+----------+
|2020-03-05 00:00:00|Saudi Arabia Refi...|       34.9|   Energy|  2030|   192.3375|  6339.375|
|2020-03-05 00:00:00|Saudi Arabian Oil...|       33.0|   Energy|  2222|   192.3375|  6339.375|
|2020-03-05 00:00:00|Rabigh Refining a...|       14.8|   Energy|  2380|   192.3375|  6339.375|
|2020-03-05 00:00:00|National Shipping...|      31.15|   Energy|  4030|   192.3375|  6339.375|
|2020-03-05 00:00:00|Aldrees Petroleum...|       60.1|   Energy|  4200|   192.3375|  6339.375|
|2020-03-05 00:00:00|Takween Advanced ...|       7.52|Materials|  1201|   192.3375|  6339.375|
|2020-03-05 00:00:00|Middle East Paper...|      13.54|Materials|  1202|   192.3375|  6339.375|
|2020-03-05 00:00:00|Basic Chemical In...|       2

In [9]:
spark

In [10]:
import pandas as pd
import numpy as np

# Assuming stocks_brent_gold_df is your DataFrame
stocks_brent_gold_df['Stock_Price_Diff'] = stocks_brent_gold_df['Stock_Price'].diff()
stocks_brent_gold_df['Price_Change'] = np.where(stocks_brent_gold_df['Stock_Price_Diff'] > 0, 'Up', 'Down')
stocks_brent_gold_df.drop('Stock_Price_Diff', axis=1, inplace=True)

# Show the updated DataFrame
print(stocks_brent_gold_df)


             Date                                          name  Stock_Price  \
0      2020-03-05                   Saudi Arabia Refineries Co.        34.90   
1      2020-03-05                         Saudi Arabian Oil Co.        33.00   
2      2020-03-05         Rabigh Refining and Petrochemical Co.        14.80   
3      2020-03-05     National Shipping Company of Saudi Arabia        31.15   
4      2020-03-05  Aldrees Petroleum and Transport Services Co.        60.10   
...           ...                                           ...          ...   
107717 2003-01-09                             Qassim Cement Co.        38.50   
107718 2003-01-09                  Southern Province Cement Co.        53.25   
107719 2003-01-09                              Yanbu Cement Co.        28.20   
107720 2003-01-09                   Eastern Province Cement Co.        42.50   
107721 2003-01-09                              Tabuk Cement Co.        20.75   

           Sector  symbol  Brent_Price 

In [ ]:
stocks_brent_gold_df['Stock_Price_Diff'] = stocks_brent_gold_df['Stock_Price'] - stocks_brent_gold_df['Stock_Price'].shift(1)

# Create a new column indicating whether the stock price increased or decreased
stocks_brent_gold_df['Price_Change'] = np.where(stocks_brent_gold_df['Stock_Price_Diff'] > 0, 'Up', 'Down')

# Drop the temporary column
stocks_brent_gold_df.drop('Stock_Price_Diff', axis=1, inplace=True)

# Show the updated DataFrame
print(stocks_brent_gold_df)


In [16]:
# create a string indexer for the name column
name_indexer = StringIndexer(inputCol="name", outputCol="nameIndex")
stocks_brent_gold_df_spark = name_indexer.fit(stocks_brent_gold_df_spark).transform(stocks_brent_gold_df_spark)


IllegalArgumentException: requirement failed: Output column nameIndex already exists.

In [14]:
# create a string indexer for the name column
name_indexer = StringIndexer(inputCol="name", outputCol="nameIndex")
stocks_brent_gold_df_spark = name_indexer.fit(stocks_brent_gold_df_spark).transform(stocks_brent_gold_df_spark)

# create a string indexer for the Sector column
sector_indexer = StringIndexer(inputCol="Sector", outputCol="SectorIndex")
stocks_brent_gold_df_spark = sector_indexer.fit(stocks_brent_gold_df_spark).transform(stocks_brent_gold_df_spark)

# create a string indexer for the symbol column
symbol_indexer = StringIndexer(inputCol="symbol", outputCol="symbolIndex")
stocks_brent_gold_df_spark = symbol_indexer.fit(stocks_brent_gold_df_spark).transform(stocks_brent_gold_df_spark)

# create a vector assembler for the features
assembler = VectorAssembler(inputCols=["nameIndex", "Stock_Price", "SectorIndex", "symbolIndex", "Brent_Price", "Gold_Price"], outputCol="features")
data = assembler.transform(stocks_brent_gold_df_spark)
data.show()

IllegalArgumentException: requirement failed: Output column nameIndex already exists.

In [15]:
stocks_brent_gold_df_spark.show()

+-------------------+--------------------+-----------+---------+------+-----------+----------+---------+-----------+-----------+
|               Date|                name|Stock_Price|   Sector|symbol|Brent_Price|Gold_Price|nameIndex|SectorIndex|symbolIndex|
+-------------------+--------------------+-----------+---------+------+-----------+----------+---------+-----------+-----------+
|2020-03-05 00:00:00|Saudi Arabia Refi...|       34.9|   Energy|  2030|   192.3375|  6339.375|     18.0|        1.0|       18.0|
|2020-03-05 00:00:00|Saudi Arabian Oil...|       33.0|   Energy|  2222|   192.3375|  6339.375|     46.0|        1.0|       46.0|
|2020-03-05 00:00:00|Rabigh Refining a...|       14.8|   Energy|  2380|   192.3375|  6339.375|     28.0|        1.0|       28.0|
|2020-03-05 00:00:00|National Shipping...|      31.15|   Energy|  4030|   192.3375|  6339.375|      1.0|        1.0|        5.0|
|2020-03-05 00:00:00|Aldrees Petroleum...|       60.1|   Energy|  4200|   192.3375|  6339.375|   

In [17]:
# create a vector assembler for the features
assembler = VectorAssembler(inputCols=["nameIndex", "Stock_Price", "SectorIndex", "symbolIndex", "Brent_Price", "Gold_Price"], outputCol="features")
data = assembler.transform(stocks_brent_gold_df_spark)
data.show()

+-------------------+--------------------+-----------+---------+------+-----------+----------+---------+-----------+-----------+--------------------+
|               Date|                name|Stock_Price|   Sector|symbol|Brent_Price|Gold_Price|nameIndex|SectorIndex|symbolIndex|            features|
+-------------------+--------------------+-----------+---------+------+-----------+----------+---------+-----------+-----------+--------------------+
|2020-03-05 00:00:00|Saudi Arabia Refi...|       34.9|   Energy|  2030|   192.3375|  6339.375|     18.0|        1.0|       18.0|[18.0,34.9,1.0,18...|
|2020-03-05 00:00:00|Saudi Arabian Oil...|       33.0|   Energy|  2222|   192.3375|  6339.375|     46.0|        1.0|       46.0|[46.0,33.0,1.0,46...|
|2020-03-05 00:00:00|Rabigh Refining a...|       14.8|   Energy|  2380|   192.3375|  6339.375|     28.0|        1.0|       28.0|[28.0,14.8,1.0,28...|
|2020-03-05 00:00:00|National Shipping...|      31.15|   Energy|  4030|   192.3375|  6339.375|      

In [18]:
# split the data into training and testing data
splits = data.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1]




In [21]:
# create a decision tree classifier
dt = DecisionTreeClassifier(labelCol="SectorIndex", featuresCol="features", maxBins=50)


In [22]:
# train the model
model = dt.fit(train)



In [23]:
# make predictions
predictions = model.transform(test)



In [24]:
# select the columns to display
predictions.select("prediction", "SectorIndex", "features").show()



+----------+-----------+--------------------+
|prediction|SectorIndex|            features|
+----------+-----------+--------------------+
|       0.0|        0.0|[0.0,20.04,0.0,0....|
|       0.0|        0.0|[25.0,20.98,0.0,2...|
|       0.0|        0.0|[2.0,81.1,0.0,2.0...|
|       0.0|        0.0|[41.0,12.48,0.0,4...|
|       0.0|        0.0|[26.0,53.66,0.0,2...|
|       0.0|        0.0|[44.0,40.85,0.0,4...|
|       0.0|        0.0|[29.0,24.38,0.0,2...|
|       0.0|        0.0|[31.0,9.96,0.0,31...|
|       1.0|        1.0|[1.0,28.45,1.0,5....|
|       0.0|        0.0|[40.0,8.82,0.0,40...|
|       0.0|        0.0|[13.0,39.35,0.0,1...|
|       1.0|        1.0|[28.0,20.78,1.0,2...|
|       0.0|        0.0|[2.0,81.0,0.0,2.0...|
|       0.0|        0.0|[3.0,124.8,0.0,1....|
|       0.0|        0.0|[6.0,61.8,0.0,6.0...|
|       0.0|        0.0|[24.0,27.76,0.0,2...|
|       0.0|        0.0|[17.0,18.8,0.0,17...|
|       0.0|        0.0|[9.0,24.8,0.0,9.0...|
|       0.0|        0.0|[38.0,8.36

In [27]:
# evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="SectorIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % accuracy)



Test Accuracy = 1 


In [28]:
# display the decision tree model
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_d012ba7ccdef, depth=1, numNodes=3, numClasses=2, numFeatures=6
  If (feature 0 in {0.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,19.0,20.0,21.0,22.0,24.0,25.0,26.0,27.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0})
   Predict: 0.0
  Else (feature 0 not in {0.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,19.0,20.0,21.0,22.0,24.0,25.0,26.0,27.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0})
   Predict: 1.0



In [32]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.maxBins, [50, 64]) \
    .build()

# Define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="SectorIndex", predictionCol="prediction", metricName="accuracy")

# Define the cross-validator
crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)  # Use 3 folds

# Train the model
cvModel = crossval.fit(train)

# Make predictions on the test data
predictions = cvModel.transform(test)

# Evaluate the model
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

# Get the best model
bestModel = cvModel.bestModel
print("Best Max Depth:", bestModel.getMaxDepth())
print("Best Max Bins:", bestModel.getMaxBins())


24/03/27 04:07:50 WARN CacheManager: Asked to cache already cached data.
24/03/27 04:07:50 WARN CacheManager: Asked to cache already cached data.


Accuracy: 1.0
Best Max Depth: 5
Best Max Bins: 50


In [33]:
# Use the best model from the cross-validation process
bestModel = cvModel.bestModel

# Make predictions on the test data using the best model
predictions = bestModel.transform(test)

# Evaluate the model's performance on the test data
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


IllegalArgumentException: label does not exist. Available: Date, name, Stock_Price, Sector, symbol, Brent_Price, Gold_Price, nameIndex, SectorIndex, symbolIndex, features, rawPrediction, probability, prediction

24/03/27 04:41:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 966891 ms exceeds timeout 120000 ms
24/03/27 04:41:05 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/27 04:57:05 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

# 3. Comparative Study


# 4. Depth of Results Analysis

# 5. Evaluation Measures

# 6. Threats to Validity